# Object Detection Code for Railway Track Faults and Breaks: From video to images

## System Performance Lab, Virginia Tech

## Notebook purpose

**Goal.** Develop a Python Notebook to extract images for the train, val and test sets.

**Data layout for the images.**
```
data/
├── train/
│   ├── fail/
│   └── ok/     
├── val/
│   ├── fail/
│   └── ok/
└── test/        
    ├── fail/
    └── ok/
```

**Organization of Python notebook.**
1. Install the packages
2. Set paths for video and images, and other specifications
3. Extract frames from video
4. Build datasets splits train, val and test


## 1. Install the packages

In [1]:
# load these packages
import os
import re
import pandas as pd
import numpy as np
import glob
import math
from pathlib import Path
import cv2
from PIL import Image   # only if you preview images
import matplotlib.pyplot as plt  # only if you preview images

## 2. Set paths for video and images, and other specifications


In [ ]:
# ---- User-configurable ----
ROOT_IN   = "/videos/"                 # folder where your videos are located
ROOT_OUT  = "/data/"          # root for extracted frames
TARGET_FPS = 5.0                # <-- extract 5 frames per second
JPEG_QUALITY = 95               # 0–100
FILENAME_PREFIX = "frame_"      # resulting image names like frame_000001.jpg

# If your videos are exactly these names, you can leave this
OK_PATTERNS   = [f"{ROOT_IN}/ok-video{i}" for i in [1,2,3]]
FAIL_PATTERNS = [f"{ROOT_IN}/fail-video{i}" for i in [1,2,3]]

# Accept with or without extension, common extensions list:
VIDEO_EXTS = ["", ".mp4", ".MP4", ".mov", ".MOV", ".mkv", ".MKV", ".avi", ".AVI"]

# Map index -> split
INDEX_TO_SPLIT = {1: "train", 2: "val", 3: "test"}

In [3]:
# Utility functions
# Creates a directory (and any missing parent directories) if it doesn’t already exist.
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# Searches for video files matching specific patterns and returns a dictionary mapping video indices (e.g., 1, 2, 3) to their file paths.
# This function is useful for batch processing videos with predictable naming (e.g., video1.mp4, video2.mp4).
def discover_video_paths(base_patterns, video_exts):
    """Return a dict { index -> filepath } for indices 1..3 if found."""
    found = {}
    for pat in base_patterns:
        m = re.search(r"video(\d+)$", pat)
        if not m:
            continue
        idx = int(m.group(1))
        candidates = []
        for ext in video_exts:
            candidates.extend(glob.glob(pat + ext))
        if candidates:
            # prefer files with an extension if both exist
            candidates = sorted(candidates, key=lambda x: (0 if Path(x).suffix else 1, x))
            found[idx] = candidates[0]
    return found

# Extracts metadata (FPS, frame count, duration) from a video file using OpenCV.
# This functions is critical for video analysis, as FPS and frame count are needed for frame-by-frame processing
def get_video_meta(path):
    cap = cv2.VideoCapture(str(path))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    if not fps or fps <= 0:
        fps = 30.0  # fallback
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration_s = frame_count / fps if (frame_count and frame_count > 0) else None
    return cap, fps, int(frame_count) if frame_count and frame_count>0 else None, duration_s

## 3. Extract frames from video


In [4]:
# Extract frames at a fixed TARGET_FPS using time-based sampling
def extract_fixed_fps(video_path, out_dir, target_fps=5.0, jpeg_quality=95, prefix="frame_"):
    """
    Extract frames every (1/target_fps) seconds using CAP_PROP_POS_MSEC seeks.
    Robust to variable frame rates.
    """
    out_dir = Path(out_dir)
    ensure_dir(out_dir)

    cap, src_fps, frame_count, duration_s = get_video_meta(video_path)

    # If no reliable duration, approximate from frames & fps
    if not duration_s or not math.isfinite(duration_s) or duration_s <= 0:
        if frame_count and frame_count > 0:
            duration_s = frame_count / src_fps
        else:
            raise RuntimeError(f"Cannot determine duration for {video_path}")

    dt = 1.0 / max(1e-6, float(target_fps))           # seconds between frames
    # timestamps: [0, dt, 2dt, ...] strictly less than duration to avoid EOF seek issues
    timestamps = np.arange(0.0, max(0.0, duration_s - 1e-6), dt)

    imwrite_params = [cv2.IMWRITE_JPEG_QUALITY, int(max(0, min(100, jpeg_quality)))]
    saved = 0

    for i, ts in enumerate(timestamps):
        cap.set(cv2.CAP_PROP_POS_MSEC, float(ts * 1000.0))
        ok, frame = cap.read()
        if not ok or frame is None:
            # fallback: try reading the next frame if seek failed
            ok2, frame2 = cap.read()
            if not ok2 or frame2 is None:
                continue
            frame = frame2

        out_path = out_dir / f"{prefix}{i:06d}.jpg"
        if not cv2.imwrite(str(out_path), frame, imwrite_params):
            print(f"Warning: failed to write {out_path}")
        else:
            saved += 1

    cap.release()
    # we make sure to return the video path, frames per second (fps), duration in seconds, number of frames, target fps, frames saved and output folder
    return {
        "video_path": str(video_path),
        "src_fps": float(src_fps),
        "duration_s": float(duration_s),
        "frame_count_src": int(frame_count) if frame_count is not None else None,
        "target_fps": float(target_fps),
        "frames_saved": int(saved),
        "out_dir": str(out_dir),
    }

## 4. Build datasets splits train, val and test

In [5]:
# Build dataset splits from ok/fail and video index (1=train, 2=val, 3=test)

# Determines the output directory where processed frames (or results) for a specific video should be saved, based on the video’s label (e.g., "ok" or "fail") and its index (e.g., 1, 2, or 3).
def target_dir_for(label: str, idx: int) -> Path:
    split = INDEX_TO_SPLIT.get(idx)
    if split is None:
        raise ValueError(f"Unexpected video index {idx}; expected 1, 2, or 3.")
    return Path(ROOT_OUT) / split / label

# Processes all videos associated with a specific label (e.g., "ok" or "fail") by:
# (i) Discovering their file paths.
# (ii) Extracting frames at a fixed FPS.
# (iii) Saving the frames to the correct output directory.
# (iv) Collecting metadata (e.g., FPS, frame count, output paths) for each video.
def process_label_group(label: str, base_patterns):
    found = discover_video_paths(base_patterns, VIDEO_EXTS)
    summaries = []
    for idx, vpath in sorted(found.items()):
        out_dir = target_dir_for(label, idx)
        info = extract_fixed_fps(
            vpath, out_dir,
            target_fps=TARGET_FPS,
            jpeg_quality=JPEG_QUALITY,
            prefix=FILENAME_PREFIX
        )
        info.update({"label": label, "index": idx, "split": INDEX_TO_SPLIT[idx]})
        summaries.append(info)
    return summaries

ok_summary   = process_label_group("ok",   OK_PATTERNS)
fail_summary = process_label_group("fail", FAIL_PATTERNS)

In [6]:
print("== OK videos ==")
print(ok_summary)
print("\n== FAIL videos ==")
print(fail_summary)

== OK videos ==
[{'video_path': '/content/drive/MyDrive/VV/Videos//ok-video1.mp4', 'src_fps': 30.021556574298213, 'duration_s': 19.45268888888889, 'frame_count_src': 584, 'target_fps': 5.0, 'frames_saved': 98, 'out_dir': '/content/drive/MyDrive/VV/Tryout images/train/ok', 'label': 'ok', 'index': 1, 'split': 'train'}, {'video_path': '/content/drive/MyDrive/VV/Videos//ok-video2.mp4', 'src_fps': 30.02141605555195, 'duration_s': 8.560555555555556, 'frame_count_src': 257, 'target_fps': 5.0, 'frames_saved': 43, 'out_dir': '/content/drive/MyDrive/VV/Tryout images/val/ok', 'label': 'ok', 'index': 2, 'split': 'val'}, {'video_path': '/content/drive/MyDrive/VV/Videos//ok-video3.mp4', 'src_fps': 29.91604546396205, 'duration_s': 9.493233333333334, 'frame_count_src': 284, 'target_fps': 5.0, 'frames_saved': 48, 'out_dir': '/content/drive/MyDrive/VV/Tryout images/test/ok', 'label': 'ok', 'index': 3, 'split': 'test'}]

== FAIL videos ==
[{'video_path': '/content/drive/MyDrive/VV/Videos//fail-video1.mp4